In [1]:
import sys; sys.path.append('../..')
import random
import pyzx as zx
import os
import pickle
import time

In [2]:
import logging

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
def sliding_window(elements, window_size):
    if len(elements) < window_size:
        return [elements]
    
    for i in range(len(elements) - window_size + 1):
        yield elements[i:i+window_size]

### Heuristic simplification
- When simplifying ZX-diagrams with T-spiders, simplification routines like full_reduce lead to a very high two-qubit gate count.
- When using heuristic-based approaches we can circumvent the problem to some extent leading to better overall circuit cost after optimization

In [4]:
random.seed(1342)
g = zx.generate.cliffordT(qubits=10, depth=500, p_t=0.3, p_cnot=0.5)
c = zx.Circuit.from_graph(g)
print(c.stats())

Circuit  on 10 qubits with 500 gates.
        132 is the T-count
        368 Cliffords among which
        260 2-qubit gates (260 CNOT, 0 other) and
        0 Hadamard gates.


In [5]:
c = zx.optimize.basic_optimization(c.split_phase_gates())
g = c.to_graph()
g_tele = zx.simplify.teleport_reduce(g)
g_tele.track_phases = False
print(zx.Circuit.from_graph(g).split_phase_gates().stats())

Circuit  on 10 qubits with 464 gates.
        56 is the T-count
        408 Cliffords among which
        252 2-qubit gates (248 CNOT, 4 other) and
        46 Hadamard gates.


In [6]:
g_full = g_tele.copy()
zx.simplify.full_reduce(g_full)
print(zx.extract_circuit(g_full.copy()).stats())

Circuit  on 10 qubits with 439 gates.
        56 is the T-count
        383 Cliffords among which
        287 2-qubit gates (67 CNOT, 220 other) and
        90 Hadamard gates.


In [7]:
applied_matches_la = []

for la in range(0,4):
    print("Lookahead:", la)
    start_time = time.time()
    g_greedy_la = g_tele.copy()
    iterations, applied_matches = zx.simplify.greedy_simp(g_greedy_la, lookahead=la)
    applied_matches_la.append(applied_matches)
    print(zx.extract_circuit(g_greedy_la.copy()).stats())
    print("Time:", time.time()-start_time)
    print("---------------------------------")

Lookahead: 0


INFO:root:Applied matches: [{'match': (5, (694, 695), -2), 'match type': 'pivot'}, {'match': (5, (700, 701), -2), 'match type': 'pivot'}, {'match': (5, (682, 683), -2), 'match type': 'pivot'}, {'match': (5, (688, 689), -2), 'match type': 'pivot'}, {'match': (5, (670, 671), -2), 'match type': 'pivot'}, {'match': (5, (676, 677), -2), 'match type': 'pivot'}, {'match': (5, (664, 665), -2), 'match type': 'pivot'}, {'match': (5, (571, 576), -2), 'match type': 'pivot'}, {'match': (5, (658, 659), -2), 'match type': 'pivot'}, {'match': (4, (365, 366), -2), 'match type': 'pivot'}, {'match': (4, (465, 479), -2), 'match type': 'pivot'}, {'match': (3, (637, 640), -2), 'match type': 'pivot'}, {'match': (3, (76, 95), -2), 'match type': 'pivot'}, {'match': (3, (644, 652), -2), 'match type': 'pivot'}, {'match': (2, (457, 473), -2), 'match type': 'pivot'}, {'match': (2, (657, 660), -2), 'match type': 'pivot'}, {'match': (2, (690, 693), -2), 'match type': 'pivot'}, {'match': (2, (678, 687), -2), 'match t

Circuit  on 10 qubits with 603 gates.
        56 is the T-count
        547 Cliffords among which
        241 2-qubit gates (3 CNOT, 238 other) and
        232 Hadamard gates.
Time: 0.4652247428894043
---------------------------------
Lookahead: 1


INFO:root:Applied matches: [{'match': (5, (694, 695), -2), 'match type': 'pivot'}, {'match': (5, (700, 701), -2), 'match type': 'pivot'}, {'match': (5, (682, 683), -2), 'match type': 'pivot'}, {'match': (5, (688, 689), -2), 'match type': 'pivot'}, {'match': (5, (670, 671), -2), 'match type': 'pivot'}, {'match': (5, (676, 677), -2), 'match type': 'pivot'}, {'match': (5, (664, 665), -2), 'match type': 'pivot'}, {'match': (5, (571, 576), -2), 'match type': 'pivot'}, {'match': (5, (658, 659), -2), 'match type': 'pivot'}, {'match': (4, (365, 366), -2), 'match type': 'pivot'}, {'match': (4, (465, 479), -2), 'match type': 'pivot'}, {'match': (3, (637, 640), -2), 'match type': 'pivot'}, {'match': (3, (76, 95), -2), 'match type': 'pivot'}, {'match': (3, (644, 652), -2), 'match type': 'pivot'}, {'match': (2, (457, 473), -2), 'match type': 'pivot'}, {'match': (2, (657, 660), -2), 'match type': 'pivot'}, {'match': (2, (690, 693), -2), 'match type': 'pivot'}, {'match': (2, (678, 687), -2), 'match t

Circuit  on 10 qubits with 591 gates.
        56 is the T-count
        535 Cliffords among which
        233 2-qubit gates (6 CNOT, 227 other) and
        228 Hadamard gates.
Time: 3.5066921710968018
---------------------------------
Lookahead: 2


INFO:root:Applied matches: [{'match': (5, (694, 695), -2), 'match type': 'pivot'}, {'match': (5, (700, 701), -2), 'match type': 'pivot'}, {'match': (5, (682, 683), -2), 'match type': 'pivot'}, {'match': (5, (688, 689), -2), 'match type': 'pivot'}, {'match': (5, (670, 671), -2), 'match type': 'pivot'}, {'match': (5, (676, 677), -2), 'match type': 'pivot'}, {'match': (5, (664, 665), -2), 'match type': 'pivot'}, {'match': (5, (571, 576), -2), 'match type': 'pivot'}, {'match': (5, (658, 659), -2), 'match type': 'pivot'}, {'match': (4, (365, 366), -2), 'match type': 'pivot'}, {'match': (4, (465, 479), -2), 'match type': 'pivot'}, {'match': (3, (637, 640), -2), 'match type': 'pivot'}, {'match': (3, (76, 95), -2), 'match type': 'pivot'}, {'match': (3, (644, 652), -2), 'match type': 'pivot'}, {'match': (2, (457, 473), -2), 'match type': 'pivot'}, {'match': (2, (657, 660), -2), 'match type': 'pivot'}, {'match': (2, (690, 693), -2), 'match type': 'pivot'}, {'match': (2, (678, 687), -2), 'match t

Circuit  on 10 qubits with 600 gates.
        56 is the T-count
        544 Cliffords among which
        238 2-qubit gates (2 CNOT, 236 other) and
        231 Hadamard gates.
Time: 14.292771339416504
---------------------------------
Lookahead: 3


INFO:root:Applied matches: [{'match': (5, (694, 695), -2), 'match type': 'pivot'}, {'match': (5, (700, 701), -2), 'match type': 'pivot'}, {'match': (5, (682, 683), -2), 'match type': 'pivot'}, {'match': (5, (688, 689), -2), 'match type': 'pivot'}, {'match': (5, (670, 671), -2), 'match type': 'pivot'}, {'match': (5, (676, 677), -2), 'match type': 'pivot'}, {'match': (5, (664, 665), -2), 'match type': 'pivot'}, {'match': (5, (571, 576), -2), 'match type': 'pivot'}, {'match': (5, (658, 659), -2), 'match type': 'pivot'}, {'match': (4, (365, 366), -2), 'match type': 'pivot'}, {'match': (4, (465, 479), -2), 'match type': 'pivot'}, {'match': (3, (637, 640), -2), 'match type': 'pivot'}, {'match': (3, (76, 95), -2), 'match type': 'pivot'}, {'match': (3, (644, 652), -2), 'match type': 'pivot'}, {'match': (2, (457, 473), -2), 'match type': 'pivot'}, {'match': (2, (657, 660), -2), 'match type': 'pivot'}, {'match': (2, (690, 693), -2), 'match type': 'pivot'}, {'match': (2, (678, 687), -2), 'match t

Circuit  on 10 qubits with 600 gates.
        56 is the T-count
        544 Cliffords among which
        238 2-qubit gates (2 CNOT, 236 other) and
        231 Hadamard gates.
Time: 41.90266752243042
---------------------------------


In [8]:
first_item = applied_matches_la[0]

for index, applied_matches in enumerate(applied_matches_la[1:], start=1):
    differences = [i for i in range(min(len(first_item), len(applied_matches))) if first_item[i] != applied_matches[i]]
    # differences = [i for i, element in enumerate(first_item) if element not in applied_matches] + [i for i, element in enumerate(applied_matches) if element not in first_item]
    if differences:
        first_diff_index = differences[0]
        print("Items in list 0 beginning at " + str(first_diff_index) + " :", first_item[first_diff_index:first_diff_index+index+1])
        print("Items in list " + str(index) + " beginning at " + str(first_diff_index) + " :", applied_matches[first_diff_index:first_diff_index+index+1])
        print("")

Items in list 0 beginning at 27 : [{'match': (1, (444, 445), -2), 'match type': 'pivot'}, {'match': (1, (390, 410), -2), 'match type': 'pivot'}]
Items in list 1 beginning at 27 : [{'match': (1, (157, 170), -2), 'match type': 'pivot'}, {'match': (2, (150, 171), -2), 'match type': 'pivot'}]

Items in list 0 beginning at 26 : [{'match': (2, (393, 397), -2), 'match type': 'pivot'}, {'match': (1, (444, 445), -2), 'match type': 'pivot'}, {'match': (1, (390, 410), -2), 'match type': 'pivot'}]
Items in list 2 beginning at 26 : [{'match': (1, (157, 170), -2), 'match type': 'pivot'}, {'match': (2, (150, 171), -2), 'match type': 'pivot'}, {'match': (2, (393, 397), -2), 'match type': 'pivot'}]

Items in list 0 beginning at 25 : [{'match': (2, (124, 138), -2), 'match type': 'pivot'}, {'match': (2, (393, 397), -2), 'match type': 'pivot'}, {'match': (1, (444, 445), -2), 'match type': 'pivot'}, {'match': (1, (390, 410), -2), 'match type': 'pivot'}]
Items in list 3 beginning at 25 : [{'match': (1, (157

In [9]:
g_nu = g_tele.copy()
zx.simplify.greedy_simp_neighbors(g_nu)
print(zx.extract_circuit(g_nu.copy()).stats())

Circuit  on 10 qubits with 586 gates.
        56 is the T-count
        530 Cliffords among which
        228 2-qubit gates (2 CNOT, 226 other) and
        221 Hadamard gates.


In [10]:
g_sim_n = g_tele.copy()
zx.simplify.sim_anneal_simp_neighbors(g_sim_n)
print(zx.extract_circuit(g_sim_n.copy()).stats())

final num edges:  480
Circuit  on 10 qubits with 610 gates.
        56 is the T-count
        554 Cliffords among which
        230 2-qubit gates (2 CNOT, 228 other) and
        245 Hadamard gates.


In [11]:
g_sim = g_tele.copy()
zx.simplify.sim_anneal_simp(g_sim)
print(zx.extract_circuit(g_sim.copy()).stats())

Circuit  on 10 qubits with 600 gates.
        56 is the T-count
        544 Cliffords among which
        248 2-qubit gates (8 CNOT, 240 other) and
        214 Hadamard gates.
